<a href="https://colab.research.google.com/github/sadiashoaib415/Quantum_Computer/blob/main/Monte_Carlo_Qiskit_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit qiskit-aer qiskit-algorithms qiskit-finance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 26.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.9 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp312-cp312-linux_x86_64.whl size=567860 sha256=d308d0b46

In [8]:
import numpy as np
from qiskit_aer import AerSimulator
from qiskit_algorithms import AmplitudeEstimation
from qiskit.primitives import StatevectorSampler
from qiskit_finance.applications.estimation import EuropeanCallPricing
from qiskit_finance.circuit.library import LogNormalDistribution

# 1. Financial Parameters
num_uncertainty_qubits = 3
spot_price = 2.0
strike_price = 2.0
volatility = 0.30
risk_free_rate = 0.05
maturity = 40 / 365

# 2. Setup the Mathematical Model
# Calculate mean and variance for the Log-Normal Distribution
mean = (risk_free_rate - 0.5 * volatility**2) * maturity + np.log(spot_price)
variance = volatility**2 * maturity
stddev = np.sqrt(variance)

# Define the price range we care about (from $0 to $4)
low = 0
high = 4
bounds = (low, high)

# 3. Build the Uncertainty Model
uncertainty_model = LogNormalDistribution(
    num_uncertainty_qubits,
    mu=mean,
    sigma=stddev**2,
    bounds=bounds
)

# 4. Create the Pricing Application
# We pass the 'bounds' here to satisfy the requirement of your library version
european_call = EuropeanCallPricing(
    num_state_qubits=num_uncertainty_qubits,
    strike_price=strike_price,
    rescaling_factor=0.25,
    uncertainty_model=uncertainty_model,
    bounds=bounds
)

# 5. Run the Quantum Algorithm
sampler = StatevectorSampler()
ae = AmplitudeEstimation(num_eval_qubits=3, sampler=sampler)

# Run estimation
problem = european_call.to_estimation_problem()
result = ae.estimate(problem)

# 6. Interpret and Print
interpreted_value = european_call.interpret(result)

print(f"--- Quantum Finance Result ---")
print(f"Estimated Option Price: ${interpreted_value:.4f}")
print(f"Confidence Interval:    {result.confidence_interval}")

--- Quantum Finance Result ---
Estimated Option Price: $1.0000
Confidence Interval:    (np.float64(0.16482192570762563), np.float64(0.23058691035730972))


Updated Code

In [10]:
from qiskit_finance.circuit.library import LogNormalDistribution
from qiskit_algorithms import IterativeAmplitudeEstimation, EstimationProblem
from qiskit_finance.applications.estimation import EuropeanCallPricing

# 1. Higher Resolution Parameters
num_uncertainty_qubits = 5 # 32 price points (much better than 8)
spot_price = 2.0
strike_price = 2.0
volatility = 0.30
risk_free_rate = 0.05
maturity = 40 / 365

# 2. Setup Distribution
mean = (risk_free_rate - 0.5 * volatility**2) * maturity + np.log(spot_price)
stddev = np.sqrt(volatility**2 * maturity)
# We set bounds at +/- 3 standard deviations for 99% coverage
low = np.exp(mean - 3 * stddev)
high = np.exp(mean + 3 * stddev)

uncertainty_model = LogNormalDistribution(num_uncertainty_qubits, mu=mean, sigma=stddev**2, bounds=(low, high))

# 3. Create Pricing App
european_call = EuropeanCallPricing(
    num_state_qubits=num_uncertainty_qubits,
    strike_price=strike_price,
    rescaling_factor=0.25,
    uncertainty_model=uncertainty_model,
    bounds=(low, high)
)

# 4. Use "Iterative" AE (More robust than standard AE)
sampler = StatevectorSampler()
iae = IterativeAmplitudeEstimation(
    epsilon_target=0.01, # Target 1% accuracy
    alpha=0.05,
    sampler=sampler
)

# 5. Execute
problem = european_call.to_estimation_problem()
iae_result = iae.estimate(problem)
final_price = european_call.interpret(iae_result)

print(f"Quantum Price: ${final_price:.4f}")
print(f"Black-Scholes: $0.0846")

Quantum Price: $0.0886
Black-Scholes: $0.0846
